In [7]:
import ee
import time

# Authenticate and initialize
ee.Authenticate()
ee.Initialize()

# India bounds
india_bounds = ee.Geometry.Rectangle([68, 6, 98, 36])

# ------------------------------------
# 1. Create 2×2° tiles over India
# ------------------------------------
def create_tiles(step=2):
    lon0, lat0, lon1, lat1 = 68, 6, 98, 36
    tiles = []
    tile_id = 0
    lon = lon0
    while lon < lon1:
        lat = lat0
        while lat < lat1:
            geom = ee.Geometry.Rectangle([lon, lat, lon + step, lat + step])
            tile = ee.Feature(geom, {'tile_id': f'tile_{tile_id}'})
            tiles.append(tile)
            tile_id += 1
            lat += step
        lon += step
    return tiles

tiles = create_tiles()
print(f"Total tiles: {len(tiles)}")

# ------------------------------------
# 2. GEDI Image Collection
# ------------------------------------
gedi = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY").filterDate('2019-04-01', '2022-12-31')
rh_keys = ['rh0','rh5','rh10','rh15','rh20','rh25','rh30','rh85','rh90','rh95','rh100']
max_shots = 100  # maximum GEDI shots allowed before sampling

# ------------------------------------
# 3. Export loop over tiles
# ------------------------------------
for tile in tiles:
    tile_id = tile.get('tile_id').getInfo()
    geom = tile.geometry()

    print(f"\nProcessing tile: {tile_id}")

    # Filter GEDI for the tile
    gedi_tile = gedi.filterBounds(geom).select(rh_keys)

    # Count number of GEDI images (shots)
    shot_count = gedi_tile.size().getInfo()
    print(f"  GEDI shots in tile: {shot_count}")

    # Sample if too many images
    if shot_count > max_shots:
        print(f"  Sampling down to {max_shots} GEDI shots")
        gedi_tile = gedi_tile.sort("system:time_start").limit(max_shots)

    # Scale RH bands from cm to meters
    def scale_rh(img):
        scaled = img.multiply(0.01)
        return scaled.copyProperties(img, img.propertyNames())

    gedi_tile_scaled = gedi_tile.map(scale_rh)

    # Mean image
    mean_img = gedi_tile_scaled.mean()

    # Reduce RH bands over the tile
    stats = mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=25,
        bestEffort=True,
        maxPixels=1e9
    )

    # Attach stats to tile feature
    feature_with_stats = tile.set(stats)

    # Wrap as FeatureCollection
    result_fc = ee.FeatureCollection([feature_with_stats])

    # Export to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=result_fc,
        description=f'GEDI_RH_{tile_id}',
        folder='India_GEDI_Tiles',
        fileNamePrefix=f'{tile_id}_GEDI_RH',
        fileFormat='CSV'
    )
    task.start()
    print(f"  Export started for {tile_id}")

    time.sleep(2)  # Delay to reduce risk of server throttling


Total tiles: 225

Processing tile: tile_0
  GEDI shots in tile: 90
  Export started for tile_0

Processing tile: tile_1
  GEDI shots in tile: 45
  Export started for tile_1

Processing tile: tile_2
  GEDI shots in tile: 90
  Export started for tile_2

Processing tile: tile_3
  GEDI shots in tile: 90
  Export started for tile_3

Processing tile: tile_4
  GEDI shots in tile: 45
  Export started for tile_4

Processing tile: tile_5
  GEDI shots in tile: 90
  Export started for tile_5

Processing tile: tile_6
  GEDI shots in tile: 90
  Export started for tile_6

Processing tile: tile_7
  GEDI shots in tile: 45
  Export started for tile_7

Processing tile: tile_8
  GEDI shots in tile: 90
  Export started for tile_8

Processing tile: tile_9
  GEDI shots in tile: 90
  Export started for tile_9

Processing tile: tile_10
  GEDI shots in tile: 45
  Export started for tile_10

Processing tile: tile_11
  GEDI shots in tile: 90
  Export started for tile_11

Processing tile: tile_12
  GEDI shots in t

  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_86

Processing tile: tile_87
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_87

Processing tile: tile_88
  GEDI shots in tile: 90
  Export started for tile_88

Processing tile: tile_89
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_89

Processing tile: tile_90
  GEDI shots in tile: 90
  Export started for tile_90

Processing tile: tile_91
  GEDI shots in tile: 45
  Export started for tile_91

Processing tile: tile_92
  GEDI shots in tile: 90
  Export started for tile_92

Processing tile: tile_93
  GEDI shots in tile: 90
  Export started for tile_93

Processing tile: tile_94
  GEDI shots in tile: 45
  Export started for tile_94

Processing tile: tile_95
  GEDI shots in tile: 90
  Export started for tile_95

Processing tile: tile_96
  GEDI shots in tile: 90
  Export started for tile_96

Processing tile: tile_97
  GEDI shots in


Processing tile: tile_171
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_171

Processing tile: tile_172
  GEDI shots in tile: 90
  Export started for tile_172

Processing tile: tile_173
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_173

Processing tile: tile_174
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_174

Processing tile: tile_175
  GEDI shots in tile: 90
  Export started for tile_175

Processing tile: tile_176
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_176

Processing tile: tile_177
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_177

Processing tile: tile_178
  GEDI shots in tile: 90
  Export started for tile_178

Processing tile: tile_179
  GEDI shots in tile: 180
  Sampling down to 100 GEDI shots
  Export started for tile_179

Processing tile: tile_180
  GEDI shots in tile: 90


In [ ]:
import ee
import time
from datetime import datetime, timedelta

# Authenticate and initialize
ee.Authenticate()
ee.Initialize()

# India bounds
india_bounds = ee.Geometry.Rectangle([68, 6, 98, 36])

# Create 2x2 degree tiles
def create_tiles(step=2):
    lon0, lat0, lon1, lat1 = 68, 6, 98, 36
    tiles = []
    tile_id = 0
    lon = lon0
    while lon < lon1:
        lat = lat0
        while lat < lat1:
            geom = ee.Geometry.Rectangle([lon, lat, lon + step, lat + step])
            tile = ee.Feature(geom, {'tile_id': f'tile_{tile_id}'})
            tiles.append(tile)
            tile_id += 1
            lat += step
        lon += step
    return tiles

tiles = create_tiles()
print(f"Total tiles: {len(tiles)}")

# GEDI collection and RH bands
gedi = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")
rh_keys = ['rh0','rh5','rh10','rh15','rh20','rh25','rh30','rh85','rh90','rh95','rh100']

# Monthly date range
start_date = datetime(2019, 4, 1)
end_date = datetime(2022, 12, 31)

def month_iterator(start, end):
    current = start
    while current <= end:
        yield current
        next_month = current.month % 12 + 1
        next_year = current.year + (current.month // 12)
        current = datetime(next_year, next_month, 1)

# Loop over each tile and month
for tile in tiles:
    tile_id = tile.get('tile_id').getInfo()
    geom = tile.geometry()
    print(f"\nProcessing tile: {tile_id}")

    for month_start in month_iterator(start_date, end_date):
        month_end = (month_start + timedelta(days=31)).replace(day=1) - timedelta(days=1)
        month_str = month_start.strftime('%Y-%m')

        print(f"  Month: {month_str}")

        # Filter GEDI for the month and tile
        gedi_tile = gedi.filterBounds(geom) \
                        .filterDate(month_start.strftime('%Y-%m-%d'), month_end.strftime('%Y-%m-%d')) \
                        .select(rh_keys)

        shot_count = gedi_tile.size().getInfo()
        print(f"    GEDI shots: {shot_count}")

        if shot_count == 0:
            print(f"    Skipping (no data)")
            continue

        # Limit shots if too many
        if shot_count > 100:
            print("    Sampling to 100")
            gedi_tile = gedi_tile.sort("system:time_start").limit(100)

        # Scale RH values
        def scale_rh(img):
            return img.multiply(0.01).copyProperties(img, img.propertyNames())
        
        gedi_tile_scaled = gedi_tile.map(scale_rh)

        # Mean image
        mean_img = gedi_tile_scaled.mean()

        # Reduce over tile
        stats = mean_img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=25,
            bestEffort=True,
            maxPixels=1e9
        )

        # Add tile_id and month to feature
        feature = tile.set(stats).set({'month': month_str})

        # Export as one-feature FeatureCollection
        result_fc = ee.FeatureCollection([feature])

        task = ee.batch.Export.table.toDrive(
            collection=result_fc,
            description=f'GEDI_RH_{tile_id}_{month_str}',
            folder='India_GEDI_Monthly',
            fileNamePrefix=f'{tile_id}_GEDI_RH_{month_str}',
            fileFormat='CSV'
        )
        task.start()
        print(f"    Export started for {tile_id}, {month_str}")

        time.sleep(2)  # Avoid throttling


Total tiles: 225

Processing tile: tile_0
  Month: 2019-04
    GEDI shots: 2
    Export started for tile_0, 2019-04
  Month: 2019-05
    GEDI shots: 2
    Export started for tile_0, 2019-05
  Month: 2019-06
    GEDI shots: 2
    Export started for tile_0, 2019-06
  Month: 2019-07
    GEDI shots: 2
    Export started for tile_0, 2019-07
  Month: 2019-08
    GEDI shots: 2
    Export started for tile_0, 2019-08
  Month: 2019-09
    GEDI shots: 2
    Export started for tile_0, 2019-09
  Month: 2019-10
    GEDI shots: 2
    Export started for tile_0, 2019-10
  Month: 2019-11
    GEDI shots: 2
    Export started for tile_0, 2019-11
  Month: 2019-12
    GEDI shots: 2
    Export started for tile_0, 2019-12
  Month: 2020-01
    GEDI shots: 2
    Export started for tile_0, 2020-01
  Month: 2020-02
    GEDI shots: 2
    Export started for tile_0, 2020-02
  Month: 2020-03
    GEDI shots: 2
    Export started for tile_0, 2020-03
  Month: 2020-04
    GEDI shots: 2
    Export started for tile_0, 2020

    Export started for tile_2, 2020-11
  Month: 2020-12
    GEDI shots: 2
    Export started for tile_2, 2020-12
  Month: 2021-01
    GEDI shots: 2
    Export started for tile_2, 2021-01
  Month: 2021-02
    GEDI shots: 2
    Export started for tile_2, 2021-02
  Month: 2021-03
    GEDI shots: 2
    Export started for tile_2, 2021-03
  Month: 2021-04
    GEDI shots: 2
    Export started for tile_2, 2021-04
  Month: 2021-05
    GEDI shots: 2
    Export started for tile_2, 2021-05
  Month: 2021-06
    GEDI shots: 2
    Export started for tile_2, 2021-06
  Month: 2021-07
    GEDI shots: 2
    Export started for tile_2, 2021-07
  Month: 2021-08
    GEDI shots: 2
    Export started for tile_2, 2021-08
  Month: 2021-09
    GEDI shots: 2
    Export started for tile_2, 2021-09
  Month: 2021-10
    GEDI shots: 2
    Export started for tile_2, 2021-10
  Month: 2021-11
    GEDI shots: 2
    Export started for tile_2, 2021-11
  Month: 2021-12
    GEDI shots: 2
    Export started for tile_2, 2021-12

  Month: 2022-08
    GEDI shots: 1
    Export started for tile_4, 2022-08
  Month: 2022-09
    GEDI shots: 1
    Export started for tile_4, 2022-09
  Month: 2022-10
    GEDI shots: 1
    Export started for tile_4, 2022-10
  Month: 2022-11
    GEDI shots: 1
    Export started for tile_4, 2022-11
  Month: 2022-12
    GEDI shots: 1
    Export started for tile_4, 2022-12

Processing tile: tile_5
  Month: 2019-04
    GEDI shots: 2
    Export started for tile_5, 2019-04
  Month: 2019-05
    GEDI shots: 2
    Export started for tile_5, 2019-05
  Month: 2019-06
    GEDI shots: 2
    Export started for tile_5, 2019-06
  Month: 2019-07
    GEDI shots: 2
    Export started for tile_5, 2019-07
  Month: 2019-08
    GEDI shots: 2
    Export started for tile_5, 2019-08
  Month: 2019-09
    GEDI shots: 2
    Export started for tile_5, 2019-09
  Month: 2019-10
    GEDI shots: 2
    Export started for tile_5, 2019-10
  Month: 2019-11
    GEDI shots: 2
    Export started for tile_5, 2019-11
  Month: 2019

  Month: 2020-07
    GEDI shots: 1
    Export started for tile_7, 2020-07
  Month: 2020-08
    GEDI shots: 1
    Export started for tile_7, 2020-08
  Month: 2020-09
    GEDI shots: 1
    Export started for tile_7, 2020-09
  Month: 2020-10
    GEDI shots: 1
    Export started for tile_7, 2020-10
  Month: 2020-11
    GEDI shots: 1
    Export started for tile_7, 2020-11
  Month: 2020-12
    GEDI shots: 1
    Export started for tile_7, 2020-12
  Month: 2021-01
    GEDI shots: 1
    Export started for tile_7, 2021-01
  Month: 2021-02
    GEDI shots: 1
    Export started for tile_7, 2021-02
  Month: 2021-03
    GEDI shots: 1
    Export started for tile_7, 2021-03
  Month: 2021-04
    GEDI shots: 1
    Export started for tile_7, 2021-04
  Month: 2021-05
    GEDI shots: 1
    Export started for tile_7, 2021-05
  Month: 2021-06
    GEDI shots: 1
    Export started for tile_7, 2021-06
  Month: 2021-07
    GEDI shots: 1
    Export started for tile_7, 2021-07
  Month: 2021-08
    GEDI shots: 1
   

    GEDI shots: 2
    Export started for tile_9, 2022-03
  Month: 2022-04
    GEDI shots: 2
    Export started for tile_9, 2022-04
  Month: 2022-05
    GEDI shots: 2
    Export started for tile_9, 2022-05
  Month: 2022-06
    GEDI shots: 2
    Export started for tile_9, 2022-06
  Month: 2022-07
    GEDI shots: 2
    Export started for tile_9, 2022-07
  Month: 2022-08
    GEDI shots: 2
    Export started for tile_9, 2022-08
  Month: 2022-09
    GEDI shots: 2
    Export started for tile_9, 2022-09
  Month: 2022-10
    GEDI shots: 2
    Export started for tile_9, 2022-10
  Month: 2022-11
    GEDI shots: 2
    Export started for tile_9, 2022-11
  Month: 2022-12
    GEDI shots: 2
    Export started for tile_9, 2022-12

Processing tile: tile_10
  Month: 2019-04
    GEDI shots: 1
    Export started for tile_10, 2019-04
  Month: 2019-05
    GEDI shots: 1
    Export started for tile_10, 2019-05
  Month: 2019-06
    GEDI shots: 1
    Export started for tile_10, 2019-06
  Month: 2019-07
    GEDI 

...

    Export started for tile_196, 2021-10
  Month: 2021-11
    GEDI shots: 2
    Export started for tile_196, 2021-11
  Month: 2021-12
    GEDI shots: 2
    Export started for tile_196, 2021-12
  Month: 2022-01
    GEDI shots: 2
    Export started for tile_196, 2022-01
  Month: 2022-02
    GEDI shots: 2
    Export started for tile_196, 2022-02
  Month: 2022-03
    GEDI shots: 2
    Export started for tile_196, 2022-03
  Month: 2022-04
    GEDI shots: 2
    Export started for tile_196, 2022-04
  Month: 2022-05
    GEDI shots: 2
    Export started for tile_196, 2022-05
  Month: 2022-06
    GEDI shots: 2
    Export started for tile_196, 2022-06
  Month: 2022-07
    GEDI shots: 2
    Export started for tile_196, 2022-07
  Month: 2022-08
    GEDI shots: 2
    Export started for tile_196, 2022-08
  Month: 2022-09
    GEDI shots: 2
    Export started for tile_196, 2022-09
  Month: 2022-10
    GEDI shots: 2
    Export started for tile_196, 2022-10
  Month: 2022-11
    GEDI shots: 2
    Export s

    GEDI shots: 2
    Export started for tile_199, 2019-06
  Month: 2019-07
    GEDI shots: 2
    Export started for tile_199, 2019-07
  Month: 2019-08
    GEDI shots: 2
    Export started for tile_199, 2019-08
  Month: 2019-09
    GEDI shots: 2
    Export started for tile_199, 2019-09
  Month: 2019-10
    GEDI shots: 2
    Export started for tile_199, 2019-10
  Month: 2019-11
    GEDI shots: 2
    Export started for tile_199, 2019-11
  Month: 2019-12
    GEDI shots: 2
    Export started for tile_199, 2019-12
  Month: 2020-01
    GEDI shots: 2
    Export started for tile_199, 2020-01
  Month: 2020-02
    GEDI shots: 2
    Export started for tile_199, 2020-02
  Month: 2020-03
    GEDI shots: 2
    Export started for tile_199, 2020-03
  Month: 2020-04
    GEDI shots: 2
    Export started for tile_199, 2020-04
  Month: 2020-05
    GEDI shots: 2
    Export started for tile_199, 2020-05
  Month: 2020-06
    GEDI shots: 2
    Export started for tile_199, 2020-06
  Month: 2020-07
    GEDI sho

    Export started for tile_201, 2020-11
  Month: 2020-12
    GEDI shots: 4
    Export started for tile_201, 2020-12
  Month: 2021-01
    GEDI shots: 4
    Export started for tile_201, 2021-01
  Month: 2021-02
    GEDI shots: 4
    Export started for tile_201, 2021-02
  Month: 2021-03
    GEDI shots: 4
    Export started for tile_201, 2021-03
  Month: 2021-04
    GEDI shots: 4
    Export started for tile_201, 2021-04
  Month: 2021-05
    GEDI shots: 4
    Export started for tile_201, 2021-05
  Month: 2021-06
    GEDI shots: 4
    Export started for tile_201, 2021-06
  Month: 2021-07
    GEDI shots: 4
    Export started for tile_201, 2021-07
  Month: 2021-08
    GEDI shots: 4
    Export started for tile_201, 2021-08
  Month: 2021-09
    GEDI shots: 4
    Export started for tile_201, 2021-09
  Month: 2021-10
    GEDI shots: 4
    Export started for tile_201, 2021-10
  Month: 2021-11
    GEDI shots: 4
    Export started for tile_201, 2021-11
  Month: 2021-12
    GEDI shots: 4
    Export s

  Month: 2022-05
    GEDI shots: 4
    Export started for tile_203, 2022-05
  Month: 2022-06
    GEDI shots: 4
    Export started for tile_203, 2022-06
  Month: 2022-07
    GEDI shots: 4
    Export started for tile_203, 2022-07
  Month: 2022-08
    GEDI shots: 4
    Export started for tile_203, 2022-08
  Month: 2022-09
    GEDI shots: 4
    Export started for tile_203, 2022-09
  Month: 2022-10
    GEDI shots: 4
    Export started for tile_203, 2022-10
  Month: 2022-11
    GEDI shots: 4
    Export started for tile_203, 2022-11
  Month: 2022-12
    GEDI shots: 4
    Export started for tile_203, 2022-12

Processing tile: tile_204
  Month: 2019-04
    GEDI shots: 4
    Export started for tile_204, 2019-04
  Month: 2019-05
    GEDI shots: 4
    Export started for tile_204, 2019-05
  Month: 2019-06
    GEDI shots: 4
    Export started for tile_204, 2019-06
  Month: 2019-07
    GEDI shots: 4
    Export started for tile_204, 2019-07
  Month: 2019-08
    GEDI shots: 4
    Export started for til

  Month: 2020-01
    GEDI shots: 4
    Export started for tile_206, 2020-01
  Month: 2020-02
    GEDI shots: 4
    Export started for tile_206, 2020-02
  Month: 2020-03
    GEDI shots: 4
    Export started for tile_206, 2020-03
  Month: 2020-04
    GEDI shots: 4
    Export started for tile_206, 2020-04
  Month: 2020-05
    GEDI shots: 4
    Export started for tile_206, 2020-05
  Month: 2020-06
    GEDI shots: 4
    Export started for tile_206, 2020-06
  Month: 2020-07
    GEDI shots: 4
    Export started for tile_206, 2020-07
  Month: 2020-08
    GEDI shots: 4
    Export started for tile_206, 2020-08
  Month: 2020-09
    GEDI shots: 4
    Export started for tile_206, 2020-09
  Month: 2020-10
    GEDI shots: 4
    Export started for tile_206, 2020-10
  Month: 2020-11
    GEDI shots: 4
    Export started for tile_206, 2020-11
  Month: 2020-12
    GEDI shots: 4
    Export started for tile_206, 2020-12
  Month: 2021-01
    GEDI shots: 4
    Export started for tile_206, 2021-01
  Month: 202

In [6]:
import ee
import time

ee.Initialize()

# Your tile creation function stays the same
def create_tiles(step=2):
    lon0, lat0, lon1, lat1 = 68, 6, 98, 36
    tiles = []
    tile_id = 0
    lon = lon0
    while lon < lon1:
        lat = lat0
        while lat < lat1:
            geom = ee.Geometry.Rectangle([lon, lat, lon + step, lat + step])
            tile = ee.Feature(geom, {'tile_id': f'tile_{tile_id}'})
            tiles.append(tile)
            tile_id += 1
            lat += step
        lon += step
    return tiles

tiles = create_tiles()

# Your helper functions remain unchanged:
def add_gcvi(img):
    gcvi = img.select('B8').divide(img.select('B3')).subtract(1).rename('GCVI')
    return img.addBands(gcvi)

def add_time(img):
    t = ee.Image.constant(ee.Date(img.get('system:time_start')).difference(ee.Date(start), 'year')).rename('t').float()
    return img.addBands(t)

def harmonic_regression(imgcol, band):
    harmonics = [1, 2, 3]
    def make_harmonics(img):
        t = img.select('t')
        terms = [ee.Image.constant(1).rename('constant')]
        for k in harmonics:
            omega = 2 * 3.1415926 * k
            terms.append(t.multiply(omega).cos().rename(f'cos_{k}'))
            terms.append(t.multiply(omega).sin().rename(f'sin_{k}'))
        return img.addBands(ee.Image.cat(terms))

    imgcol = imgcol.map(make_harmonics)
    independents = ['constant'] + [f'cos_{k}' for k in harmonics] + [f'sin_{k}' for k in harmonics]

    regression = imgcol.select(independents + [band]).reduce(ee.Reducer.linearRegression(len(independents), 1))
    coefficients = regression.select('coefficients').arrayProject([0]).arrayFlatten([independents])
    return coefficients

bands_to_fit = {
    'NIR': 'B8',
    'SWIR1': 'B11',
    'SWIR2': 'B12',
    'RED_EDGE4': 'B8A',
    'GCVI': 'GCVI'
}

start = '2019-04-01'
end   = '2022-12-31'

s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start, end)
s2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate(start, end)

for tile in tiles:
    tile_id = tile.get('tile_id').getInfo()
    geom = tile.geometry()

    print(f"\nProcessing tile: {tile_id}")

    sr_tile = s2_sr.filterBounds(geom)
    clouds_tile = s2_clouds.filterBounds(geom)

    joined = ee.Join.inner().apply(sr_tile, clouds_tile,
        ee.Filter.equals(leftField='system:index', rightField='system:index'))

    s2_with_clouds = ee.ImageCollection(joined.map(
        lambda f: ee.Image(f.get('primary')).addBands(
            ee.Image(f.get('secondary')).select('probability').rename('cloud_prob'))
    ))

    s2_filtered = s2_with_clouds.map(lambda img: img.updateMask(img.select('cloud_prob').lt(65)))
    s2_filtered = s2_filtered.map(add_gcvi).map(add_time)

    count = s2_filtered.size().getInfo()
    print(f"  Number of images: {count}")
    if count == 0:
        print("  Skipping (no images).")
        continue

    harmonic_outputs = []
    for name, band in bands_to_fit.items():
        coeffs = harmonic_regression(s2_filtered, band)
        renamed = coeffs.rename([f'{name}_{b}' for b in coeffs.bandNames().getInfo()])
        harmonic_outputs.append(renamed)

    harmonic_img = ee.Image.cat(harmonic_outputs).clip(geom)

    # ----------- New: Sample the harmonic_img over a grid of points inside tile -----------

    # Define a function to create a grid of points inside the geometry:
    def create_point_grid(geometry, scale=10000):  # 10km grid spacing (~0.1 degree)
        # Get bounds of the tile
        bounds = geometry.bounds().getInfo()['coordinates'][0]
        lon_min = bounds[0][0]
        lat_min = bounds[0][1]
        lon_max = bounds[2][0]
        lat_max = bounds[2][1]

        points = []
        lon = lon_min
        while lon < lon_max:
            lat = lat_min
            while lat < lat_max:
                points.append(ee.Feature(ee.Geometry.Point([lon, lat])))
                lat += scale / 111320.0  # approx degrees for given meters
            lon += scale / 111320.0
        return ee.FeatureCollection(points)

    points_fc = create_point_grid(geom, scale=10000)  # you can increase or decrease scale for density

    # Sample harmonic coefficients at points
    sampled_points = harmonic_img.sampleRegions(
        collection=points_fc,
        scale=10,
        geometries=True
    )

    # Export the FeatureCollection as CSV
    task = ee.batch.Export.table.toDrive(
        collection=sampled_points,
        description=f'{tile_id}_harmonic_coeffs_csv',
        folder='S2_Harmonics_CSV',
        fileFormat='CSV'
    )
    task.start()
    print(f"  CSV export started for {tile_id}")

    time.sleep(2)  # prevent rate limits



Processing tile: tile_0
  Number of images: 1029
  CSV export started for tile_0

Processing tile: tile_1
  Number of images: 0
  Skipping (no images).

Processing tile: tile_2
  Number of images: 0
  Skipping (no images).

Processing tile: tile_3
  Number of images: 0
  Skipping (no images).

Processing tile: tile_4
  Number of images: 0
  Skipping (no images).

Processing tile: tile_5
  Number of images: 0
  Skipping (no images).

Processing tile: tile_6
  Number of images: 0
  Skipping (no images).

Processing tile: tile_7
  Number of images: 4107
  CSV export started for tile_7

Processing tile: tile_8
  Number of images: 4657
  CSV export started for tile_8

Processing tile: tile_9
  Number of images: 4687
  CSV export started for tile_9

Processing tile: tile_10
  Number of images: 6214
  CSV export started for tile_10

Processing tile: tile_11
  Number of images: 5253
  CSV export started for tile_11

Processing tile: tile_12
  Number of images: 4145
  CSV export started for ti

  CSV export started for tile_98

Processing tile: tile_99
  Number of images: 5581
  CSV export started for tile_99

Processing tile: tile_100
  Number of images: 5322
  CSV export started for tile_100

Processing tile: tile_101
  Number of images: 5264
  CSV export started for tile_101

Processing tile: tile_102
  Number of images: 3843
  CSV export started for tile_102

Processing tile: tile_103
  Number of images: 3878
  CSV export started for tile_103

Processing tile: tile_104
  Number of images: 4716
  CSV export started for tile_104

Processing tile: tile_105
  Number of images: 1659
  CSV export started for tile_105

Processing tile: tile_106
  Number of images: 2092
  CSV export started for tile_106

Processing tile: tile_107
  Number of images: 0
  Skipping (no images).

Processing tile: tile_108
  Number of images: 1532
  CSV export started for tile_108

Processing tile: tile_109
  Number of images: 2416
  CSV export started for tile_109

Processing tile: tile_110
  Number 

  Number of images: 4440
  CSV export started for tile_196

Processing tile: tile_197
  Number of images: 4411
  CSV export started for tile_197

Processing tile: tile_198
  Number of images: 4436
  CSV export started for tile_198

Processing tile: tile_199
  Number of images: 4766
  CSV export started for tile_199

Processing tile: tile_200
  Number of images: 6236
  CSV export started for tile_200

Processing tile: tile_201
  Number of images: 6297
  CSV export started for tile_201

Processing tile: tile_202
  Number of images: 5191
  CSV export started for tile_202

Processing tile: tile_203
  Number of images: 4450
  CSV export started for tile_203

Processing tile: tile_204
  Number of images: 4837
  CSV export started for tile_204

Processing tile: tile_205
  Number of images: 6449
  CSV export started for tile_205

Processing tile: tile_206
  Number of images: 6240
  CSV export started for tile_206

Processing tile: tile_207
  Number of images: 5534
  CSV export started for tile

# Interpolation and Count of Sequences

In [8]:
import pandas as pd
import re

# --- Load the CSV file ---
df = pd.read_csv("India_GEDI_Monthly-20250603T135546Z-1-001/India_GEDI_Data.csv")  # Replace with your actual file path
df['tile_id'] = df['tile_id'].astype(str)

# --- Sort by tile_id (and assume data is already in correct temporal order) ---
df = df.sort_values('tile_id')

# --- Interpolate rh95 within each tile ---
df['rh95_interp'] = df.groupby('tile_id')['rh95'].transform(lambda x: x.interpolate(limit_direction='both'))

# --- Create boolean mask for rh95 > 2.5 ---
df['rh95_gt_2.5'] = df['rh95_interp'] > 2.5

# --- Function to find max consecutive True ---
def max_consecutive_true(series):
    max_count = count = 0
    for val in series:
        if val:
            count += 1
            max_count = max(max_count, count)
        else:
            count = 0
    return max_count

# --- Group by tile_id and compute max consecutive ---
counts = df.groupby('tile_id')['rh95_gt_2.5'].apply(max_consecutive_true).reset_index(name='max_consecutive_rh95_gt_2.5')

# --- Ensure all tile_ids present ---
all_tiles = pd.DataFrame({'tile_id': df['tile_id'].unique()})
result = all_tiles.merge(counts, on='tile_id', how='left').fillna(0)
result['max_consecutive_rh95_gt_2.5'] = result['max_consecutive_rh95_gt_2.5'].astype(int)

# --- Natural sorting function for tile IDs like 'tile_0', 'tile_10', 'tile_100' ---
def tile_sort_key(t):
    m = re.search(r'(\d+)$', t)
    return int(m.group(1)) if m else t

# --- Sort results naturally by tile number ---
result = result.sort_values(by='tile_id', key=lambda col: col.map(tile_sort_key)).reset_index(drop=True)

# --- Print all tiles and their max consecutive counts ---
for _, row in result.iterrows():
    print(f"{row['tile_id']}: {row['max_consecutive_rh95_gt_2.5']}")


tile_0: 0
tile_1: 0
tile_2: 0
tile_3: 0
tile_4: 0
tile_5: 0
tile_6: 0
tile_7: 0
tile_8: 0
tile_9: 0
tile_10: 0
tile_11: 0
tile_12: 0
tile_13: 0
tile_14: 0
tile_15: 0
tile_16: 0
tile_17: 0
tile_18: 0
tile_19: 0
tile_20: 0
tile_21: 0
tile_22: 0
tile_23: 0
tile_24: 0
tile_25: 0
tile_26: 0
tile_27: 0
tile_28: 0
tile_29: 0
tile_30: 0
tile_31: 0
tile_32: 0
tile_33: 0
tile_34: 0
tile_35: 0
tile_36: 0
tile_37: 0
tile_38: 0
tile_39: 0
tile_40: 0
tile_41: 0
tile_42: 0
tile_43: 0
tile_44: 0
tile_45: 0
tile_46: 0
tile_47: 0
tile_48: 0
tile_49: 0
tile_50: 0
tile_51: 0
tile_52: 0
tile_53: 0
tile_54: 0
tile_55: 0
tile_56: 0
tile_57: 0
tile_58: 0
tile_59: 0
tile_60: 0
tile_61: 0
tile_62: 0
tile_63: 0
tile_64: 0
tile_65: 0
tile_66: 0
tile_67: 0
tile_68: 0
tile_69: 0
tile_70: 0
tile_71: 0
tile_72: 0
tile_73: 0
tile_74: 0
tile_75: 0
tile_76: 0
tile_77: 0
tile_78: 0
tile_79: 0
tile_80: 0
tile_81: 0
tile_82: 0
tile_83: 0
tile_84: 0
tile_85: 0
tile_86: 0
tile_87: 0
tile_88: 0
tile_89: 0
tile_90: 0
tile_91: 

In [9]:
max_rh95 = df['rh95'].max()
print("Maximum rh95 value in the file:", max_rh95)

Maximum rh95 value in the file: 0.2930137232907219
